In [50]:
import json
from time import sleep

import pandas as pd
import pyautogui as pg
import pyperclip
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [51]:
def selenium_run():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    url = 'https://code4biz.slack.com/'

    # 暗黙的な待機
    driver.implicitly_wait(5)
    driver.get(url)
    driver.maximize_window()
    return driver

In [52]:
def auto_login(driver):
    # ログインページ処理
    form = driver.find_element(by=By.CSS_SELECTOR, value='#signin_form')
    login_mail = form.find_element(by=By.NAME, value='email')
    login_passwd = form.find_element(by=By.NAME, value='password')

    # 認証情報
    with open('src/auth.json') as f:
        auth = json.load(f)

    my_mail = auth['my_mail']
    my_pass = auth['my_pass']

    # ログイン情報初期化
    login_mail.clear()
    login_passwd.clear()

    login_mail.send_keys(my_mail)
    login_passwd.send_keys(my_pass)

    # サインインボタンを押す
    sleep(0.5)
    btn = form.find_element(by=By.CSS_SELECTOR, value='#signin_btn')
    btn.click()
    sleep(2)

    # Slackアプリ起動確認メッセージのキャンセル
    pg.press('enter', presses=1, interval=0.1)
    sleep(1)

    # ブラウザでSlack起動
    pg.press('tab', presses=2, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)

    # SlackのWebページが完全に立ち上がるまで5秒待機
    sleep(8)

In [53]:
# PyAutoGUIで日本語入力も対応する
def write_jp(text):
    sleep(0.5)
    pyperclip.copy(text)
    pg.hotkey('ctrl', 'v')


# course_link列をハイパーリンク化
def make_clickable(course_link):
    return f'<a target="_blank" href="{course_link}">{course_link}</a>'


def jump_to_search_box():
    # 検索フィルターで期間指定
    pg.press('tab', presses=12, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    sleep(1)


def display_smaller():
    # 検索結果表示画面の表示を小さくする（一度の表示できる件数を増やす）
    sleep(1)
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    pg.hotkey('ctrl', '-')
    sleep(1)

In [54]:
def to_search_yesterday():
    jump_to_search_box()

    # 日付検索を昨日にする
    pg.press('tab', presses=4, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('down', presses=3, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

    # 検索条件決定
    pg.press('tab', presses=5, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

In [55]:
def to_search_period(start, end):
    jump_to_search_box()

    # 範囲指定
    pg.press('tab', presses=4, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('up', presses=1, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    # 開始日時を”YYYY-MM-DD”形式で入力
    write_jp(start)

    sleep(0.5)
    pg.press('tab', presses=1, interval=0.1)

    # 終了日時を”YYYY-MM-DD”形式で入力
    write_jp(end)

    # 検索範囲の保存ボタンを押す
    sleep(0.5)
    pg.hotkey('shift', 'tab')
    pg.hotkey('shift', 'tab')
    pg.hotkey('shift', 'tab')
    pg.press('enter', presses=1, interval=0.1)

    # 検索開始（上記保存することでコメント欄にカーソルアクティブしている）
    sleep(0.2)
    pg.press('tab', presses=15, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

    print(start, end)

In [56]:
def to_search_date(search_day):
    jump_to_search_box()

    # 日時指定
    pg.press('tab', presses=4, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('up', presses=4, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

    # 開始日時を”YYYY-MM-DD”形式で入力
    write_jp(search_day)

    # 保存ボタンを押す
    sleep(0.5)
    pg.hotkey('shift', 'tab')
    pg.hotkey('shift', 'tab')
    pg.press('enter', presses=1, interval=0.1)

    # 検索開始（上記保存することでコメント欄にカーソルアクティブしている）
    sleep(0.2)
    pg.press('tab', presses=15, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

    print(f'検索日:{search_day}')

In [57]:
driver = selenium_run()
auto_login(driver)

# 昨日のみ
# to_search_yesterday()

# 期間指定
# start = '2022-03-13'
# end = '2022-03-13'
# to_search_period(start=start, end=end)

# 日時指定
search_day = '2022-03-13'
to_search_date(search_day)

sleep(2)
# 検索件数の取得
search_results_count = driver.find_element(by=By.CLASS_NAME, value='c-tabs__tab_count').text
print(f'検索結果数：{search_results_count}')

# 並べ替え：古い→新
pg.press('tab', presses=16, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
pg.press('down', presses=3, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
# 全件表示
pg.press('tab', presses=1, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
pg.press('down', presses=3, interval=0.2)
pg.press('enter', presses=1, interval=0.1)

# 表示を小さくする
display_smaller()

# 一番下まで移動
pg.press('tab', presses=2, interval=0.2)
pg.press('pagedown', presses=10, interval=0.2)
sleep(2)

data = []

# メッセージ重複有り（取得後整形予定）
for i in range(3):  # 3回までスクロール
    message_groups = driver.find_elements(by=By.CLASS_NAME, value='c-message_group')
    print(f'メッセージ数:{len(message_groups)}')

    for j, message in enumerate(message_groups):
        date = message.find_element(by=By.CLASS_NAME, value='c-message_group__header_date').text
        timestamp = message.find_element(by=By.CLASS_NAME, value='c-timestamp__label').text
        channel_name = message.find_element(by=By.CLASS_NAME, value='p-deprecated_channel_name__text').text
        sender_name = message.find_element(by=By.CLASS_NAME, value='c-message__sender').text
        try:
            text_section = message.find_element(by=By.CLASS_NAME, value='p-rich_text_section').text
            try:
                # リスト表記をしているメッセージへの対応
                lists = message.find_elements(by=By.TAG_NAME, value='li')
                if lists is not None:
                    for list in lists:
                        text_section += list.text + '\n'
            except:
                pass
        except:
            text_section = message.find_element(by=By.CLASS_NAME, value='c-search_message__body').text
            try:
                # リスト表記をしているメッセージへの対応
                lists = message.find_elements(by=By.TAG_NAME, value='li')
                if lists is not None:
                    for list in lists:
                        text_section += list.text + '\n'
            except:
                pass
        urls = message.find_elements(by=By.TAG_NAME, value='a')
        url = urls[1].get_attribute('href')
        ts = url.split('=')[-1]

        datum = {
            '検索結果数': search_results_count,
            'ts': ts,
            '投稿日': date,
            '投稿時間': timestamp,
            '投稿チャンネル': channel_name,
            '投稿者': sender_name,
            '投稿メッセージ': text_section,
            'リンク': url
        }
        data.append(datum)
        print(f'{j + 1}/{len(message_groups)}:{sender_name}さん{timestamp}投稿の、「{text_section[:10]}」から始まるメッセージ取得完了')
        # 待機時間（サイトに負荷を与えないと同時にコンテンツの読み込み待ち）
        sleep(1)

    # 先頭の要素までスクロール
    first_message = message_groups[0]
    actions = ActionChains(driver);
    actions.move_to_element(first_message);
    actions.perform();

    # 待機時間（サイトに負荷を与えないと同時にコンテンツの読み込み待ち）
    sleep(3)

df = pd.DataFrame(data)
df['リンク'] = df['リンク'].apply(make_clickable)
# df.to_excel(f'src/code4biz_slack_message_{start}_to_{end}.xlsx', index=False)
df.to_excel(f'src/{search_day}_code4biz_slack_messages.xlsx', index=False)
df

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


検索日:2022-03-13
検索結果数：19
メッセージ数:19
1/19:imanyuさん00:48投稿の、「がこのチャンネルの説」から始まるメッセージ取得完了
2/19:imanyuさん00:48投稿の、「がこのチャンネルの説」から始まるメッセージ取得完了
3/19:imanyuさん00:49投稿の、「がこのチャンネルの説」から始まるメッセージ取得完了
4/19:imanyuさん00:50投稿の、「がこのチャンネルの説」から始まるメッセージ取得完了
5/19:松坂さん07:00投稿の、「はじめまして
松坂（」から始まるメッセージ取得完了
6/19:imanyuさん10:30投稿の、「@松坂

自己紹介あ」から始まるメッセージ取得完了
7/19:imanyuさん11:45投稿の、「@here
一時的に」から始まるメッセージ取得完了
8/19:imanyuさん13:16投稿の、「@here
みなさん」から始まるメッセージ取得完了
9/19:やまもとゆうきさん17:09投稿の、「はじめまして
４０代」から始まるメッセージ取得完了
10/19:imanyuさん17:13投稿の、「@やまもとゆうき

」から始まるメッセージ取得完了
11/19:imanyuさん17:14投稿の、「@here
Slac」から始まるメッセージ取得完了
12/19:やまもとゆうきさん17:14投稿の、「こちらこそ、よろしく」から始まるメッセージ取得完了
13/19:木下和治さん18:31投稿の、「木下和治と申します。」から始まるメッセージ取得完了
14/19:imanyuさん18:33投稿の、「@kino

木下さ」から始まるメッセージ取得完了
15/19:木下和治さん18:44投稿の、「Udemyで、キカガ」から始まるメッセージ取得完了
16/19:imanyuさん18:58投稿の、「@kino

分かり」から始まるメッセージ取得完了
17/19:imanyuさん22:05投稿の、「＜ぼやき＞
私が運営」から始まるメッセージ取得完了
18/19:笠井健太さん22:23投稿の、「皆さん初めまして。
」から始まるメッセージ取得完了
19/19:imanyuさん22:24投稿の、「@笠井健太

笠井さ」から始まるメッセージ取得完了
メッセージ数:19
1/19:imanyuさん00:48投稿の、「

,検索結果数,ts,投稿日,投稿時間,投稿チャンネル,投稿者,投稿メッセージ,リンク
0,19,https://code4biz.slack.com/archives/C036NJSDVN...,3月13日,00:48,02-いまにゅに相談,imanyu,がこのチャンネルの説明を「いまにゅに何でも相談してください！！\n学習やキャリア相談、プライ...,"<a target=""_blank"" href=""https://code4biz.slac..."
1,19,https://code4biz.slack.com/archives/C036NQRBV0...,3月13日,00:48,01-講義更新情報,imanyu,がこのチャンネルの説明を「教材や運営に関する更新情報を共有するチャンネルです！」に設定しました,"<a target=""_blank"" href=""https://code4biz.slac..."
2,19,https://code4biz.slack.com/archives/C036A0NPU1...,3月13日,00:49,01-いまにゅのつぶやき,imanyu,がこのチャンネルの説明を「いまにゅがただぼやくための場所です。遠慮なくコメントしていただけれ...,"<a target=""_blank"" href=""https://code4biz.slac..."
3,19,https://code4biz.slack.com/archives/C036ZNV6ZS...,3月13日,00:50,03-自己紹介,imanyu,がこのチャンネルの説明を「Slackに入りましたら是非自己紹介をお願いします\n」に設定しました,"<a target=""_blank"" href=""https://code4biz.slac..."
4,19,1647122414.551459,3月13日,07:00,03-自己紹介,松坂,はじめまして\n松坂（まつざか）と申します。\nPython歴:独学約3ヶ月\n（上記とは別...,"<a target=""_blank"" href=""https://code4biz.slac..."
5,19,1647122414.551459,3月13日,10:30,03-自己紹介,imanyu,@松坂\n\n自己紹介ありがとうございます！\n\nおぉ、よくわからぬままDjangoを触っ...,"<a target=""_blank"" href=""https://code4biz.slac..."
6,19,https://code4biz.slack.com/archives/C02HLGD5FK...,3月13日,11:45,01-全体アナウンス,imanyu,@here\n一時的に皆さんを質問ルームから外しております\n\n追って招待いたします！！,"<a target=""_blank"" href=""https://code4biz.slac..."
7,19,https://code4biz.slack.com/archives/C036NQRBV0...,3月13日,13:16,01-講義更新情報,imanyu,@here\nみなさん、おはようございます！\n\n「code4bizの進め方」パートに動画...,"<a target=""_blank"" href=""https://code4biz.slac..."
8,19,1647158998.148279,3月13日,17:09,03-自己紹介,やまもとゆうき,はじめまして\n４０代サラリーマンのやまもとと申します。\n息子とPythonとラズパイを学...,"<a target=""_blank"" href=""https://code4biz.slac..."
9,19,1647158998.148279,3月13日,17:13,03-自己紹介,imanyu,@やまもとゆうき\n\nおぉ、ラズパイも学習されているんですね\n\n息子さんとの工作にはも...,"<a target=""_blank"" href=""https://code4biz.slac..."


In [60]:
driver = selenium_run()
auto_login(driver)

# 昨日のみ
# to_search_yesterday()

# 期間指定
# start = '2022-03-13'
# end = '2022-03-13'
# to_search_period(start=start, end=end)

# 日時指定
search_day = '2022-03-15'
to_search_date(search_day)

sleep(2)
# 検索件数の取得
search_results_count = driver.find_element(by=By.CLASS_NAME, value='c-tabs__tab_count').text
print(f'検索結果数：{search_results_count}')

# 並べ替え：古い→新
pg.press('tab', presses=16, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
pg.press('down', presses=3, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
# 全件表示
pg.press('tab', presses=1, interval=0.2)
pg.press('enter', presses=1, interval=0.1)
pg.press('down', presses=3, interval=0.2)
pg.press('enter', presses=1, interval=0.1)

# 表示を小さくする
display_smaller()

# 一番下まで移動
pg.press('tab', presses=2, interval=0.2)
pg.press('pagedown', presses=10, interval=0.2)
sleep(2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


検索日:2022-03-15
検索結果数：48


In [65]:

sleep(5)
# data = []
#
message_groups = driver.find_elements(by=By.CLASS_NAME, value='c-message_group')
urls_list  = []
for j, message in enumerate(message_groups):
    urls = message.find_elements(by=By.TAG_NAME, value='a')
    url = urls[1].get_attribute('href')
    urls_list.append(url)

    sleep(1)

    # 先頭の要素までスクロール
    first_message = message_groups[0]
    actions = ActionChains(driver);
    actions.move_to_element(first_message);
    actions.perform();
urls_list

['https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647299253977339?thread_ts=1647299253.977339',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647309648645169?thread_ts=1647299253.977339',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647310979494039?thread_ts=1647310979.494039',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647311112057179?thread_ts=1647310979.494039',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647311320869129?thread_ts=1647310979.494039',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647311340098119?thread_ts=1647310979.494039',
 'https://code4biz.slack.com/archives/C036A0NPU1H/p1647311447191059?thread_ts=1647311447.191059',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647313972753049?thread_ts=1647262858.989699',
 'https://code4biz.slack.com/archives/C036ZNV6ZS4/p1647314603421879?thread_ts=1647262858.989699',
 'https://code4biz.slack.com/archives/C036A0NPU1H/p1647314975361839?thread_ts=1647311447.191059',
 'https://code4biz.s

In [69]:
urls_list[0]
sleep(5)
driver.get(urls_list[0])

In [47]:
driver.quit()